In [57]:
import pandas as pd
import csv
import panel as pn
import matplotlib
from math import pi
from bokeh.palettes import Category20c, Category20
from bokeh.plotting import figure
from bokeh.transform import cumsum
import sys
from panel import HSpacer, Spacer

# This needs to be able to compare two tables and only pull from the auction table properties to buy
# Meaning, pulling data of 3 props from realsetateinv to compare to 1 prop from Auction.
prosp = pd.read_csv('ProspectList.csv')
comp = pd.read_csv('complist.csv')
df1 = pd.DataFrame(prosp) #Prospects
df2 = pd.DataFrame(comp) #Comps to compare with the property in question to buy
df10 = df1.filter(items=['Address2'])
df20 = df2.filter(items=['Address2'])
df1.sort_index(inplace=True)
df2.sort_index(inplace=True)
cmps = df2.loc[0:3,'Address2']
#We only need the ones that match and are more available. Just delete the #'s to test the print.
#print(cmps)
#print(df1['Address2'])

#Getting the best property from prospectlist
#headers Address	Address2	Price	Bed	Bath	Sqft	Date	Time
df11 = df1.loc[1]
df21 = df2[df2["Address2"].isin(["Sunrise, FL 33351"])]
df22 = df21[df21.Bed.isin([2])]
print(df22) #if you test this you'll see we now got a list of comps we can use.
print(df11)
#Now we just gotta add a few things, its market value(after repairs), prerepair offer, and avg repair costs
sumcomp = df22['Price'].sum()
avgcomp = sumcomp / 2
d = {}
df00 = pd.DataFrame(index=[1])
df00['MarketValue'] = avgcomp.astype(int) #assuming repairs and profit added
repairc = 40000 #assuming tons of repairs are needed (example) IN reality, an in-person inspection is needed.
df00['RepairCost'] = repairc
#df00 = df00.assign(repairC = repairc) #Another method that applies to each column too
profit =(avgcomp * 0.3) #assuming 30% profit
offer = avgcomp - profit - repairc
df00['Offer'] = offer
df00['Profit'] = profit
print(df00)

#Launch it using jupyter labs AND any help at: https://panel.holoviz.org/getting_started/index.html


                       Address           Address2   Price  Bed  Bath  Sqft  \
24  4025 N Nob Hill Rd Apt 106  Sunrise, FL 33351  220000    2     2  1074   
36   7911 NW 41st Ct Unit 7911  Sunrise, FL 33351  345000    2     2   954   

        Date      Time  
24  01/02/24  01:55AM   
36  01/02/24  01:55AM   
Address     2701 E Sunrise Lakes Dr Apt 312
Address2                  Sunrise, FL 33322
Price                                136999
Bed                                       2
Bath                                      2
Sqft                                    850
Date                               01/02/24
Time                               01:55AM 
Name: 1, dtype: object
   MarketValue  RepairCost     Offer   Profit
1       282500       40000  157750.0  84750.0


In [58]:
pn.extension()
df01 = df00[['RepairCost','Offer','Profit']]
#print(df01)
x= df01['RepairCost'].iloc[0]
y= df01['Offer'].iloc[0]
z= df01['Profit'].iloc[0]

df02 = pd.DataFrame({'cost': [x, y , z]}, index=['Repair Cost', 'Offer', 'Profit'])
#piechart = df02.plot.pie(y='cost', figsize=(8, 5), title='Cashflow Pie Summary')
#instead of plotpie use https://panel.holoviz.org/reference/panes/Bokeh.html


df02['angle'] = df02['cost']/df02['cost'].sum() * 2*pi
df02['color'] = Category20c[len(df02['cost'])]
p = figure(height=350, title="Cashflow Pie Summary", toolbar_location=None,
           tools="hover", tooltips="@index: @cost", x_range=(-0.5, 1.0))
r = p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", color='color',legend_field='index', source=df02)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None

bokeh_paneo = pn.pane.Bokeh(p, theme="dark_minimal")
bokeh_paneo

Bokeh(figure, theme='dark_minimal')

In [59]:
dfpane = pn.pane.DataFrame(df00, width=400)
dfpane

DataFrame(DataFrame, width=400)

In [60]:
dfpane22 = pn.pane.DataFrame(df22, width=500)
dfpane22

DataFrame(DataFrame, width=500)

In [61]:
dfpane11 = pn.pane.DataFrame(df11, width=300, header=False)
dfpane11

DataFrame(Series, header=False, width=300)

In [62]:
rprc = pn.widgets.FloatSlider(name="Repair Cost", start=10000, end=40000, value=40000, width= 300)
rprc

FloatSlider(end=40000, name='Repair Cost', start=10000, value=40000, width=300)

In [63]:
def card_func(value0):
    val = round((profit - rprc.value),1)
    card0 = pn.Card(
        pn.indicators.Number(value=val, default_color='black', name='Profit (adj)', format='${value}'),
        styles={'background': 'skyblue'},
        hide_header=True,
        width=300
    )
    return card0
card0s = pn.bind(card_func, value0=rprc)
card0s

<function param.reactive.bind.<locals>.wrapped(*wargs, **wkwargs)>

In [64]:
prpc = (profit / avgcomp) * 100

card11 = pn.Card(
    pn.indicators.Number(value=prpc, default_color='black', name='Profit (%)', format='{value}%'),
    styles={'background': 'skyblue'},
    hide_header=True,
    width=300
)


def card_func1(value1):
    prpc = round((((profit - rprc.value) / avgcomp) * 100),2)
    card1 = pn.Card(
        pn.indicators.Number(value=prpc, default_color='black', name='Profit (%)', format='{value}%'),
        styles={'background': 'skyblue'},
        hide_header=True,
        width=300
    )
    return card1
card1 = pn.bind(card_func1, value1=rprc)
card1

<function param.reactive.bind.<locals>.wrapped(*wargs, **wkwargs)>

In [65]:
adjv = round(((profit + offer + rprc.value)/ 1000),1)

card23 = pn.Card(
    pn.indicators.Number(value=adjv, default_color='black', name='Adjusted Value', format='${value}K'),
    styles={'background': 'skyblue'},
    hide_header=True,
    width=300
)

def card_func2(value2):
    adjv = round(((profit + offer + rprc.value)/ 1000),1)
    card2 = pn.Card(
        pn.indicators.Number(value=adjv, default_color='black', name='Adjusted Value', format='${value}K'),
        styles={'background': 'skyblue'},
        hide_header=True,
        width=300
    )
    return card2
card2 = pn.bind(card_func2, value2=rprc)
card2

<function param.reactive.bind.<locals>.wrapped(*wargs, **wkwargs)>

In [66]:
import os
import ctypes 


def exit_app(self):
    try:
        os._exit(0)
        print("You can close the Browser now.")
    except:
        print("Something went horribly wrong.")
        

button1 = pn.widgets.Button(name='',icon='browser-x', button_type='danger', button_style='solid',icon_size='2em',description='Close Dashboard.',align="center")
button1.on_click(exit_app)
button1

Button(align='center', button_type='danger', description='Close Dashboard.', icon='browser-x', icon_size='2em')

In [67]:
template = pn.template.FastListTemplate(
    title='Potential Property to Buy',
    header=pn.Row(HSpacer(),(button1),align='end'),
    sidebar=[card1, card2,rprc,card0s],
    main=[pn.Row(pn.Column(bokeh_paneo), pn.Column(dfpane11)),pn.Row(dfpane22,dfpane)]
)
# Append a layout to the main area, to demonstrate the list-like API
#template.main.append()
template.servable();